In [10]:
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
from dask.dataframe.utils import make_meta
import matplotlib.pyplot as plt 
import numpy as np  
import lsdb
import pandas as pd
from catalog_filtering import bandFilterLenient, contains_PM

In [11]:
bandList = ['G','R','I','Z','Y']
class_star = 0.95
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
check_invalid_mags = True
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,invalidMags=check_invalid_mags)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
des_gaia_xmatch = lsdb.read_hipscat('/Volumes/PSC/des_gaia_xmatch/')
min_stars = 4
k = 1
collinear_error = 0.2
pm_speed_min = 2000 #units are milliseconds per year
cone_search_rad = 20

In [12]:
pm_filter_xmatch = des_gaia_xmatch.query(f'(pmra_gaia**2 + pmdec_gaia**2) > {pm_speed_min**2}')

In [13]:
with Client():
    df = pm_filter_xmatch.compute()

df

/Users/johnpassos/Documents/LSDB/venv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49474 instead
  warnings.warn(


,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,...,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,ra_gaia,pmra_gaia,dec_gaia,pmdec_gaia,source_id_gaia,parallax_over_error_gaia,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
4613930437209751552,0.84542,0.845421,0.845413,0.845344,0.035287,19,19,19,19,19,...,-99.0,-99.0,-99.0,1.383284,5633.438088,-37.367744,-2334.721273,2306965202564744064,6359.301,1.786142
5030074573559693312,0.844564,0.845075,0.845251,0.843732,0.801461,2,2,2,2,2,...,-99.0,-99.0,-99.0,33.079599,-1762.405718,3.567385,-1852.8711,2515037264041041536,3857.4338,1.62095
9473310124119425024,0.048989,0.404167,0.99414,0.02865,0.028268,3,3,3,3,3,...,0.000554,-99.0,-99.0,53.567196,2360.592206,-49.890084,483.127504,4736655036868019072,2020.9392,2.582885
9473310124769542144,0.717914,0.984056,0.848698,0.544428,0.028628,3,3,3,3,3,...,-99.0,-99.0,-99.0,53.567196,2360.592206,-49.890084,483.127504,4736655036868019072,2020.9392,4.575273
9560201318102990848,0.125803,0.120065,0.028601,0.028627,0.028627,3,3,3,3,3,...,-99.0,-99.0,-99.0,62.611,-825.17937,-53.612997,-2415.577565,4780100658292046592,6581.664,4.490123
9560201322498621440,0.644246,0.036391,0.842178,0.788,0.034785,3,3,3,3,3,...,-99.0,-99.0,-99.0,62.611,-825.17937,-53.612997,-2415.577565,4780100658292046592,6581.664,4.672577
9695914633298182144,0.845425,0.845425,0.845406,0.845371,0.84513,22,22,18,18,22,...,-99.0,-99.0,-99.0,50.000344,3035.017316,-43.066553,726.964482,4847957293278177024,2112.3093,0.825611
9800217929181036544,0.844839,0.845014,0.845117,0.845267,0.844982,2,2,2,2,6,...,-99.0,-99.0,-99.0,5.03561,1706.746855,-64.869617,1164.959443,4900108950849461248,871.1839,3.096527
9874001828230725632,0.84536,0.029197,0.845341,0.842975,0.047746,2,3,2,2,2,...,-99.0,-99.0,-99.0,32.622946,2125.416147,-50.820906,637.975043,4937000898856156288,2016.6344,0.425698


In [51]:
df_no_dupes = df[~df['source_id_gaia'].duplicated(keep='first')]

df_no_dupes

,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,...,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,ra_gaia,pmra_gaia,dec_gaia,pmdec_gaia,source_id_gaia,parallax_over_error_gaia,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
4613930437209751552,0.84542,0.845421,0.845413,0.845344,0.035287,19,19,19,19,19,...,-99.0,-99.0,-99.0,1.383284,5633.438088,-37.367744,-2334.721273,2306965202564744064,6359.301,1.786142
5030074573559693312,0.844564,0.845075,0.845251,0.843732,0.801461,2,2,2,2,2,...,-99.0,-99.0,-99.0,33.079599,-1762.405718,3.567385,-1852.8711,2515037264041041536,3857.4338,1.62095
9473310124119425024,0.048989,0.404167,0.99414,0.02865,0.028268,3,3,3,3,3,...,0.000554,-99.0,-99.0,53.567196,2360.592206,-49.890084,483.127504,4736655036868019072,2020.9392,2.582885
9560201318102990848,0.125803,0.120065,0.028601,0.028627,0.028627,3,3,3,3,3,...,-99.0,-99.0,-99.0,62.611,-825.17937,-53.612997,-2415.577565,4780100658292046592,6581.664,4.490123
9695914633298182144,0.845425,0.845425,0.845406,0.845371,0.84513,22,22,18,18,22,...,-99.0,-99.0,-99.0,50.000344,3035.017316,-43.066553,726.964482,4847957293278177024,2112.3093,0.825611
9800217929181036544,0.844839,0.845014,0.845117,0.845267,0.844982,2,2,2,2,6,...,-99.0,-99.0,-99.0,5.03561,1706.746855,-64.869617,1164.959443,4900108950849461248,871.1839,3.096527
9874001828230725632,0.84536,0.029197,0.845341,0.842975,0.047746,2,3,2,2,2,...,-99.0,-99.0,-99.0,32.622946,2125.416147,-50.820906,637.975043,4937000898856156288,2016.6344,0.425698
9874001828230725632,0.84536,0.029197,0.845341,0.842975,0.047746,2,3,2,2,2,...,-99.0,-99.0,-99.0,32.624069,2168.886011,-50.820823,710.847727,4937000898856154624,588.3757,2.199757
10012464071594475520,0.036971,0.466729,0.943789,0.983893,0.035059,3,3,3,3,3,...,0.003134,0.002709,0.008519,11.341389,1826.373986,-33.497993,-1485.010343,5006232026455470848,166.26512,4.295977


In [24]:
def distance_to_line(line_vector, PQ):
    return np.abs(np.cross(PQ, line_vector)) / np.linalg.norm(line_vector)

def within_collinear_error(line_vector, PQ, error):
    return (distance_to_line(line_vector, PQ) <= error)

def kth_star_min_distance(group, k, cols_to_keep):
    origin_ra, origin_dec = group['RA_1'], group['DEC_1']
    ra2, dec2 = group[["RA_2", "DEC_2"]].to_numpy().T
    
    x_vals = (ra2 - origin_ra) * np.cos(np.radians(origin_dec)) * 3600
    y_vals = (dec2 - origin_dec) * 3600
    delta_coords = np.vstack((x_vals, y_vals)).T

    kth_distances = [None] * len(delta_coords)

    for i in range(len(delta_coords)):
        proj_vector = delta_coords[i]
        distances = []

        if np.all(proj_vector == 0): continue #ensures vector points to a point that is not the origin

        for j in range(len(delta_coords)):
            if np.all(delta_coords[j] == 0) or (j == i): continue
            
            distances.append(distance_to_line(proj_vector, delta_coords[j]))

        kth_distances[i] = sorted(distances)[k]
    
    group['kth_min_proj_error'] = kth_distances

    return group[cols_to_keep + ['kth_min_proj_error']]

In [60]:
def algo_found_pm(row, accuracy):
    catalog = lsdb.read_hipscat('/Volumes/PSC/des_dr2_light/', columns=des_cols).cone_search(ra=row['ra_gaia'], dec=row['dec_gaia'], radius_arcsec=cone_search_rad)
    filtered_catalog = catalog.query(query_string)
    xmatch = filtered_catalog.crossmatch(filtered_catalog, n_neighbors=100, radius_arcsec=18, suffixes=['_1', '_2'])

    xmatch_ddf = xmatch._ddf
    neighbors = xmatch_ddf.groupby('_hipscat_index')['_dist_arcsec'].count()
    neighbors -= 1 #Double counting adjustment
    neighbors.name = 'neighbors'
    xmatch_ddf = xmatch_ddf.join(neighbors, on='_hipscat_index')
    xmatch_large_groups = xmatch_ddf.query('neighbors >= 3')
    columns_to_keep = (
        [col for col in xmatch_large_groups.columns if col.endswith('_2')]
    )
    meta = make_meta(xmatch_large_groups[columns_to_keep])
    meta['kth_min_proj_error'] = 0.0
    star_groups = xmatch_large_groups.groupby('_hipscat_index').apply(kth_star_min_distance, meta=meta, k=k-1, cols_to_keep = columns_to_keep)  
    df = star_groups.compute()
    if not (df.query(f'kth_min_proj_error < {collinear_error}').empty):
        accuracy += 1
    


In [61]:
accuracy = 0
df_no_dupes.apply(func=algo_found_pm, axis=0, accuracy=accuracy)

accuracy

KeyError: 'ra_gaia'